In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

**Define Model**

In PyTorch, your models should also subclass `torch.nn.Module` class.

In this tutorial, we will build a convolutional neural network for image classification.

![alt text](https://miro.medium.com/max/2510/1*XbuW8WuRrAY5pC4t-9DZAQ.jpeg)



In [0]:
class Net(nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Conv2d and nn.Linear modules and assign them as
        member variables.
        """
        super(Net, self).__init__()
        
        num_classes = 10
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=4*4*50, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=num_classes)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        
        x = F.relu(self.conv1(x))  # [batchx1x28x28] --> [batchx20x24x24]
        x = F.max_pool2d(x, 2, 2)  # [batchx20x24x24] --> [batchx20x12x12]
        x = F.relu(self.conv2(x))  # [batchx20x12x12] --> [batchx50x8x8]
        x = F.max_pool2d(x, 2, 2)  # [batchx50x8x8] --> [batchx50x4x4]
        x = x.view(-1, 4*4*50)     # flatten 3D tensor to 1D
        x = F.relu(self.fc1(x))    # [batchx(50x4x4)] --> [batchx500]
        x = self.fc2(x)            # [batchx500] --> [batchx10]
        return F.log_softmax(x, dim=1)

In [12]:
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [13]:
print(model.conv2)
print(model.conv2.weight.shape)
print(model.conv2.weight.device)

Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
torch.Size([50, 20, 5, 5])
cpu


**Set Hyperparameters**

In [14]:
import argparse

parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=3, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')

args = parser.parse_args(args=['--save-model'])
print(args)

Namespace(batch_size=64, epochs=3, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, save_model=True, seed=1, test_batch_size=1000)


**Fix Random Seed**

In [15]:
torch.manual_seed(args.seed)

In [16]:
import torch.optim as optim

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

print(model.conv2.weight.device)

cuda:0


**Define Dataset and DataLoader**

MNIST handwritten digit dataset

![alt text](https://www.researchgate.net/profile/Steven_Young11/publication/306056875/figure/fig1/AS:393921575309346@1470929630835/Example-images-from-the-MNIST-dataset.png)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from torchvision import datasets, transforms

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

0it [00:00, ?it/s]

9920512it [00:00, 20136150.37it/s]                            


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw


32768it [00:00, 330079.79it/s]
0it [00:00, ?it/s]

Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw


1654784it [00:00, 5938665.77it/s]                           
8192it [00:00, 113537.12it/s]


Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw
Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw
Processing...
Done!


**Train Phase**

Set your model in training mode (this has any effect only on certain modules, e.g. Dropout, BatchNorm, etc.)

![alt text](https://blog.exxactcorp.com/wp-content/uploads/2019/06/tensors_flowing.gif)

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

**Test Phase**

Set your model in evaluation mode (this has any effect only on certain modules, e.g. `Dropout`, `BatchNorm`, etc.)

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

**Start Training and Testing**

In [19]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.279750
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.254422
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.185086
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.131899
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.983129
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.678403
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.371962
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.034748
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.687925
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.476121
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.484266
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.621842
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.565654
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.753579
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.408918
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.462027
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.352636
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.224936
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.234863
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.301127
Train Epoch: 1 [12800/60000 (

**Save the trained model**

A `state_dict` is simply a Python dictionary object that maps each layer to its parameter tensor.

Because `state_dict` objects are Python dictionaries, they can be easily saved, updated, altered, and restored, adding a great deal of modularity to PyTorch models and optimizers.

When saving a model for inference, it is only necessary to save the trained model’s learned parameters. Saving the model’s `state_dict` with the `torch.save()` function will give you the most flexibility for restoring the model later, which is why it is the recommended method for saving models.

(https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [20]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    
if (args.save_model):
  torch.save(model.state_dict(),"mnist_cnn.pt")

Model's state_dict:
conv1.weight 	 torch.Size([20, 1, 5, 5])
conv1.bias 	 torch.Size([20])
conv2.weight 	 torch.Size([50, 20, 5, 5])
conv2.bias 	 torch.Size([50])
fc1.weight 	 torch.Size([500, 800])
fc1.bias 	 torch.Size([500])
fc2.weight 	 torch.Size([10, 500])
fc2.bias 	 torch.Size([10])


**Load the pre-trained model**

In [21]:
new_model = Net()
new_model = new_model.to(device)

# test the model with random initialization
test(args, new_model, device, test_loader)

# load the pre-trained parameters
new_model.load_state_dict(torch.load('mnist_cnn.pt'))

# test the pre-trained model
test(args, new_model, device, test_loader)


Test set: Average loss: 2.3093, Accuracy: 1109/10000 (11%)


Test set: Average loss: 0.0515, Accuracy: 9835/10000 (98%)

